### Librerias 

In [ ]:
import pandas as pd
import ast
import json
import csv
from textblob import TextBlob
from fastapi import FastAPI

### Procesamiento de datos y conversión de JSON a CSV

#### transformar el archivo 'user_reviews.json' a 'user_reviews.csv'

In [ ]:
# función para corregir y cargar cada línea del archivo 'user_reviews.json' original
def cargar_y_corregir_linea(linea):
    # reemplaza las comillas simples con dobles y corrige valores booleanos
    linea_corregida = linea.replace("'", '"').replace('True', 'true').replace('False', 'false')
    return json.loads(linea_corregida)

# crea y abre un archivo 'user_reviews.csv' para escribir los datos
with open('user_reviews.csv', 'w', newline='', encoding='utf-8') as archivo_csv:
    escritor_csv = csv.writer(archivo_csv)
    # escribe la fila de encabezado
    encabezado = ['user_id', 'user_url', 'funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend', 'review']
    escritor_csv.writerow(encabezado)
    
    # lee el archivo 'user_reviews.json' original, corrige y procesa cada línea
    with open('user_reviews.json', 'r', encoding='utf-8') as archivo_json:
        for linea in archivo_json:
            try:
                entrada = cargar_y_corregir_linea(linea)
                user_id = entrada['user_id']
                user_url = entrada['user_url']
                # iteracion sobre cada review
                for reseña in entrada['reviews']:
                    escritor_csv.writerow([
                        user_id,
                        user_url,
                        reseña.get('funny', ''),
                        reseña.get('posted', ''),
                        reseña.get('last_edited', ''),
                        reseña.get('item_id', ''),
                        reseña.get('helpful', ''),
                        reseña.get('recommend', ''),
                        reseña.get('review', '')  # asume que quieres mantener el texto original de la review
                    ])
            except json.JSONDecodeError as e:
                print(f"Error al procesar la línea: {e}")

print("Los datos han sido convertidos y guardados en 'user_reviews.csv'.")


#### transformar el archivo 'output_steam_games.json' a 'output_steam_games.csv'

In [ ]:
# Lee el archivo JSON
df = pd.read_json('output_steam_games.json', lines=True)

# Elimina las filas vacías
df = df.dropna(how='all')

# Guarda el DataFrame en un archivo CSV
df.to_csv('output_steam_games.csv', index=False)

#### transformar el archivo 'australian_users_items.json' a 'australian_users_items.csv'

In [ ]:
# función para corregir y cargar cada línea del archivo 'australian_users_items.json' original
def cargar_y_corregir_linea(linea):
    # reemplaza las comillas simples con dobles y corrige valores booleanos
    linea_corregida = linea.replace("'", '"')
    return json.loads(linea_corregida)

# crea y abre un archivo 'australian_users_items.csv' para escribir los datos
with open('australian_users_items.csv', 'w', newline='', encoding='utf-8') as archivo_csv:
    writer = csv.writer(archivo_csv)
    
    # Escribir los nombres de las columnas
    writer.writerow(['user_id', 'user_url', 'item_id', 'item_name', 'playtime_forever', 'playtime_2weeks'])
    
    # lee el archivo 'australian_users_items.json' original, corrige y procesa cada línea
    with open('australian_users_items.json', 'r', encoding='utf-8') as archivo_json:
        for linea in archivo_json:
            try:
                entrada = cargar_y_corregir_linea(linea)
                user_id = entrada['user_id']
                user_url = entrada['user_url']
                # iteracion sobre cada items
                for articulo in entrada['items']:
                    writer.writerow([
                        user_id,
                        user_url,
                        articulo.get('item_id', ''),
                        articulo.get('item_name', ''),
                        articulo.get('playtime_forever', ''),
                        articulo.get('playtime_2weeks', ''),
                    ])
            except json.JSONDecodeError as e:
                print(f"Error al procesar la línea: {e}")

print("Los datos han sido convertidos y guardados en 'australian_users_items.csv'.")


### Análisis de sentimiento con NLP

El dataset **User_Reviews** incluye reseñas de juegos hechas por distintos usuarios. Se crea la columna `sentiment_analysis` con la siguiente escala:

- `0` si es una reseña negativa.
- `1` si es neutral o hay ausencia de reseña.
- `2` si es una reseña positiva.

Esta nueva columna reemplazará la columna `review` para facilitar el trabajo de los modelos de machine learning y el análisis de datos.


In [ ]:
# carga del archivo CSV
df = pd.read_csv('user_reviews.csv')

# función para analizar el sentimiento de la reseña
def analyze_sentiment(review):
    if pd.isna(review) or not isinstance(review, str):
        return 1  # sentimiento neutral para reseñas faltantes o no legibles
    else:
        # se emplea TextBlob para calcular la polaridad del sentimiento
        polarity = TextBlob(review).sentiment.polarity
        if polarity < 0:
            return 0  # sentimiento negativo
        elif polarity == 0:
            return 1  # sentimiento neutral
        else:
            return 2  # sentimiento positivo

# aplica la función analyze_sentiment a cada reseña y crear la nueva columna 'sentiment_analysis'
df['sentiment_analysis'] = df['review'].apply(analyze_sentiment)

# elimina la columna original 'review'
df.drop(columns=['review'], inplace=True)

# guarda el DataFrame modificado en un nuevo archivo CSV
df.to_csv('user_reviews.csv', index=False)

print("El análisis de sentimiento ha sido aplicado y guardado en 'user_reviews.csv'.")


### Transformacion de los archivos CSV a Parquet

In [ ]:

# Leer archivos CSV
user_reviews_df = pd.read_csv('user_reviews.csv')
steam_games_df = pd.read_csv('output_steam_games.csv')
australian_users_df = pd.read_csv('australian_users_items.csv')

# Escribir archivos Parquet
user_reviews_df.to_parquet('user_reviews.parquet', index=False)
steam_games_df.to_parquet('output_steam_games.parquet', index=False)
australian_users_df.to_parquet('australian_users_items.parquet', index=False)

print("Archivos transformados a formato Parquet exitosamente.")